In [3]:
# Create a saver.
saver = tf.train.Saver(...variables...)
# Launch the graph and train, saving the model every 1,000 steps.
sess = tf.Session()
for step in xrange(1000000):
    sess.run(..training_op..)
    if step % 1000 == 0:
        # Append the step number to the checkpoint name:
        saver.save(sess, 'my-model', global_step=step)

# to retrieve the saved tensorflow model:
# After this, the value of tensors like w1 and w2 has been restored and can be accessed:
with tf.Session() as sess:
  new_saver = tf.train.import_meta_graph('my_test_model-1000.meta')
  new_saver.restore(sess, tf.train.latest_checkpoint('./'))
  print(sess.run('w1:0'))

SyntaxError: invalid syntax (<ipython-input-3-f6bfdb7cb74b>, line 2)

In [1]:
import tensorflow as tf
import os
from pathlib import Path
 
#Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder("float", name="w1")
w2 = tf.placeholder("float", name="w2")
b1= tf.Variable(2.0,name="bias")
feed_dict ={w1:4,w2:8}
 
#Define a test operation that we will restore
w3 = tf.add(w1,w2)
w4 = tf.multiply(w3,b1,name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
 
#Create a saver object which will save all the variables
saver = tf.train.Saver()
 
#Run the operation by feeding input
print(sess.run(w4,feed_dict))
#Prints 24 which is sum of (w1+w2)*b1 

24.0


In [2]:
#Now, save the graph
logdir = 'C:/real_summaries'
checkpoint_file = os.path.join(Path(logdir), 'my_test_model')
print(checkpoint_file)
# print(os.path.pardir) # this print: .. (current directory)
saver.save(sess, checkpoint_file,global_step=1000)


C:\real_summaries\my_test_model


'C:\\real_summaries\\my_test_model-1000'

In [32]:
#How to access saved variable/Tensor/placeholders 
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0") # w1 is a tensor variable without value, for this you must execute sess.run(var,feed_dict={w1:4,w2:8}).
# print(w1.eval(session=tf.Session())) # this works only after a run operation.
 
## How to access saved operation
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
# print(op_to_restore.eval(session=tf.Session()))
print(Path(logdir))

C:\real_summaries


In [9]:
# to retrive the graph:
# to restore the model, add new operations and training with your own data:
# the .meta file contains the weights and gradients? YES
# how to include the hyperparameters??
checkpoint_file = os.path.join(Path(logdir), 'my_test_model-1000.meta')
print(checkpoint_file)
# with tf.Session() as sess:    
sess = tf.Session()
new_saver = tf.train.import_meta_graph(checkpoint_file)
new_saver.restore(sess, tf.train.latest_checkpoint(Path(logdir)))
print(sess.run('w1:0', {w1:10.0, w2:9.0}))

# Now, let's access and create placeholders variables and
# create feed-dict to feed new data
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}
 
#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
 
print(sess.run(op_to_restore,feed_dict))
#This will print 60 which is calculated 
#using new values of w1 and w2 and saved value of b1. 

C:\real_summaries\my_test_model-1000.meta
INFO:tensorflow:Restoring parameters from C:\real_summaries\my_test_model-1000
10.0
60.0


In [11]:
# Now, let's access and create placeholders variables and
# create feed-dict to feed new data
 
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}
 
#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
 
#Add more to the current graph
add_on_op = tf.multiply(op_to_restore,2)
 
print(sess.run(add_on_op,feed_dict))
#This will print 120.

120.0


In [ ]:
'''
# this is a real world example:
saver = tf.train.import_meta_graph('vgg.meta')
# Access the graph
graph = tf.get_default_graph()
## Prepare the feed_dict for feeding data for fine-tuning 
 
#Access the appropriate output for fine-tuning
fc7= graph.get_tensor_by_name('fc7:0')
 
#use this if you only want to change gradients of the last layer
fc7 = tf.stop_gradient(fc7) # It's an identity function
fc7_shape= fc7.get_shape().as_list()
 
new_outputs=2
weights = tf.Variable(tf.truncated_normal([fc7_shape[3], num_outputs], stddev=0.05))
biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
output = tf.matmul(fc7, weights) + biases
pred = tf.nn.softmax(output)
 
# Now, you run this with fine-tuning data in sess.run()
'''